In [1]:
import numpy as np
import math
from datetime import datetime
import random

# Sample Inputs
stops_input = [
    {"name": "Lajpat Nagar", "lat": 28.817559, "lon": 77.104537, "condition": False},
    {"name": "Dwarka Sector 10", "lat": 28.875682, "lon": 77.321606, "condition": True},

    {"name": "Sarojini Nagar", "lat": 28.688008, "lon": 76.871453, "condition": True},
    {"name": "Vivek Vihar", "lat": 28.77752, "lon": 77.124188, "condition": False},
    {"name": "Red Fort", "lat": 28.72066, "lon": 77.169159, "condition": True},
    {"name": "Janakpuri", "lat": 28.682632, "lon": 77.147544, "condition": False},
    {"name": "Dwarka Sector 21", "lat": 28.629151, "lon": 76.847819, "condition": False},
]

alternate_routes_input = {
    1: [("Bus101", 10)],  # Alternate for stop 1
    2: [("Bus102", 20)],  # Alternate for stop 2
}

total_passengers = 200

# Determine time of day (simulating "non-peak" for consistent testing)
time_of_day = "non-peak"  # Can simulate "peak" if needed

# Function Definitions
def haversine(lat1, lon1, lat2, lon2):
    R = 6371  # Earth's radius in km
    phi1, phi2 = math.radians(lat1), math.radians(lat2)
    delta_phi = math.radians(lat2 - lat1)
    delta_lambda = math.radians(lon2 - lon1)
    a = math.sin(delta_phi / 2) ** 2 + math.cos(phi1) * math.cos(phi2) * math.sin(delta_lambda / 2) ** 2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
    return R * c

def compute_distance_matrix(stops):
    num_cities = len(stops)
    distance_matrix = np.zeros((num_cities, num_cities))
    for i in range(num_cities):
        for j in range(i + 1, num_cities):
            distance = haversine(stops[i]["lat"], stops[i]["lon"], stops[j]["lat"], stops[j]["lon"])
            distance_matrix[i][j] = distance
            distance_matrix[j][i] = distance
    return distance_matrix

def calculate_route_feasibility(optimized_route, affected_stops, alternate_routes, total_passengers, time_of_day):
    impact_threshold = 25 if time_of_day == "peak" else 35
    total_affected = 0
    for stop_index, num_passengers in affected_stops.items():
        if stop_index in alternate_routes:
            alternate_available = any(ETA <= 15 for bus_id, ETA in alternate_routes[stop_index])
            if alternate_available:
                continue
        if stop_index not in optimized_route:
            total_affected += num_passengers
    percentage_affected = (total_affected / total_passengers) * 100
    if percentage_affected <= impact_threshold:
        return True
    return False

def ant_colony_optimization(
    stops,
    total_passengers,
    time_of_day,
    num_ants=50,
    num_iterations=100,
    alpha=1.2,
    beta=3.0,
    evaporation_rate=0.5,
    q=50,
    initial_pheromone=1.0,
    alternate_routes=None,
):
    num_cities = len(stops)
    distance_matrix = compute_distance_matrix(stops)
    pheromone_matrix = np.full((num_cities, num_cities), initial_pheromone)

    def calculate_probabilities(current_city, unvisited, pheromone_matrix, distance_matrix):
        distances = distance_matrix[current_city, unvisited]
        distances[distances == 0] = 1e-10
        pheromones = pheromone_matrix[current_city, unvisited] ** alpha
        distances = (1 / distances) ** beta
        probabilities = pheromones * distances
        total_prob = probabilities.sum()
        if total_prob == 0:
            probabilities = np.ones_like(probabilities) / len(probabilities)
        else:
            probabilities /= total_prob
        return probabilities

    def calculate_tour_length(tour, distance_matrix):
        return sum(distance_matrix[tour[i - 1], tour[i]] for i in range(1, len(tour)))

    def filter_stops(unvisited, stops):
        return [i for i in unvisited if not stops[i]["condition"]]

    best_tour = None
    best_distance = float("inf")

    for iteration in range(num_iterations):
        all_tours = []
        all_distances = []
        for ant in range(num_ants):
            tour = [0]
            unvisited = list(range(1, num_cities))
            while unvisited:
                unvisited = filter_stops(unvisited, stops)
                if not unvisited:
                    break
                current_city = tour[-1]
                probabilities = calculate_probabilities(
                    current_city, unvisited, pheromone_matrix, distance_matrix
                )
                next_city = np.random.choice(unvisited, p=probabilities)
                tour.append(next_city)
                unvisited.remove(next_city)
            distance = calculate_tour_length(tour, distance_matrix)
            all_tours.append(tour)
            all_distances.append(distance)
            if distance < best_distance:
                best_tour = tour
                best_distance = distance

        pheromone_matrix *= (1 - evaporation_rate)
        for tour, distance in zip(all_tours, all_distances):
            pheromone_increase = q / distance
            for i in range(len(tour) - 1):
                pheromone_matrix[tour[i], tour[i + 1]] += pheromone_increase
                pheromone_matrix[tour[i + 1], tour[i]] += pheromone_increase

    affected_stops = {
        i: random.randint(5, 20) for i in range(num_cities) if stops[i]["condition"]
    }
    route_feasible = calculate_route_feasibility(
        best_tour, affected_stops, alternate_routes, total_passengers, time_of_day
    )

    if route_feasible:
        best_tour_cities = [stops[i]["name"] for i in best_tour]
        return best_tour_cities, best_distance
    return None, float("inf")

# Running ACO
best_tour, best_distance = ant_colony_optimization(
    stops_input,
    total_passengers,
    time_of_day,
    num_ants=50,
    num_iterations=100,
    alpha=1.2,
    beta=3.0,
    evaporation_rate=0.5,
    q=50,
    initial_pheromone=1.0,
    alternate_routes=alternate_routes_input,
)

# Output Results
(best_tour, best_distance)


(['Lajpat Nagar', 'Vivek Vihar', 'Janakpuri', 'Dwarka Sector 21'],
 45.4847283657969)